#### Step 1: Prepare the data

import necessary libraries for loading the data:

nibabel for reading and writing neuroimaging data.
numpy for numerical operations and array handling.

In [1]:
import nibabel as nb
import numpy as np

This cell loads and preprocesses the brain imaging data and phenotype (demographic) information:

- Brain imaging data (cifti_data) is loaded from a .nii file.
- Network labels (network_label) are loaded and filtered to remove unspecified indices (-1).
- Binary vectors (net_6, net_7) are created for two specific networks, indicating vertex membership.
- Phenotype data (age, sex) is loaded and combined into a single array (phenotype).

In [2]:
# load brain imaging data
cifti_data = nb.load('./Dataset/HCP_WB_Tutorial_1.0/Q1-Q6_R440.All.sulc.32k_fs_LR.dscalar.nii').get_fdata(dtype=np.float32) # (replace with code to load your data)
network_labels = np.load('./Dataset/network_label_yeo7.npz')['arr_0'] # (replace with code to load your network labels)

idx = network_labels!=-1  # identify which locations should be ignored (e.g., medial wall)
network_label = network_labels[idx]  # remove labels outside idx
X = cifti_data[:,idx] # subset image (X) locations to idx

# generate binary vector for each network, 0-> specific vertex does not belong to the network, 1-> specific vertex belongs to the network.
net_7 = np.where(network_label!=7,0,1)  # create a binary vector with 1's at locations corresponding to network 7 and 0's at locations outside network 7

# Phenotype of interest (y) and covariates (Z)
y = np.load('./Dataset/R440_age.npz')['arr_0'] # replace with code to load your phenotype (e.g., age)
Z = np.load('./Dataset/R440_gender.npz')['arr_0'] # replace with code to load other covariates/confounders (e.g., sex)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Print the shapes of the processed data arrays to verify their dimensions, ensuring they are correct for subsequent analysis. In this example, the sample size is 440 and number of vertices is 58606.

In [3]:
X.shape, y.shape, Z.shape, net_7.shape

((440, 58606), (440,), (440,), (58606,))

#### Step 2: Import the NEST package after the installation.

The package can be installed via 'pip install nest-sw'

In [4]:
from NEST import nest

#### Step 3: Define the following dictionary of arguments passed to the nest method. The args can be defined as follows, assuming vertex-wise linear models will be fit to estimate local brain-phenotype associations (i.e., specifying statFun='lm' in step 4.).

- X: N x P matrix (numpy array) of P imaging features (e.g., vertices) for N participants.

- y: N-dimensional vector of phenotype of interest (i.e., testing enrichment of X-y associations).

- Z: Optional. Specify one or more covariates (matrix with N rows and q columns for q covariates). Default is NULL (no covariates to be included).

- FL: Optional (default is False). Set to True to use Freedman-Lane procedure to account for dependence between covariates in permutation.

- n_perm: Optional (default is 999, with smallest possible p-value of 1/1000).

In [5]:
args = {
    'X': X, # brain measurements (dimension N subjects x P image locations).
    'y': y, # phenotype of interest (dimension N).
    'Z': Z, # covariates (dimension (N x # number of covariates).
    'type': 'coef', # what type of test statistic to extract from linear regression model.
    'FL': False, # Not use Freedman-Lane procedure.
    'n_perm': 999 # how many permutations to use to obtain null distribution.
}

##### Step 4: Apply NEST to test enrichment of brain-phenotype associations in specified network.

In [6]:
pval,ES_obs,ES_null,_ = nest(statFun='lm', # use linear regression to get vertex-level test statistics
                             args=args, # arguments specified above (specific to statFun="lm")
                             net_maps=net_7, # list of binary indicating locations inside (1) or outside (1) network(s) of interest.
                             one_sided=True, # Determines whether the enrichment score calculation should consider only the positive alignment (True) or both directions (False).
                             seed=None, # Random seed for reproducible permutation. Default is None. 
                             )

Using the following default settings: 


args['getNull'] = True ---> defaulting to get a null distribution



Print the p-value and observed enrichment score

In [8]:
pval

0.076

Print the enrichment scores for null distribution

NEST also supports the use of customized statistical functions for calculating associations. 
This advanced feature allows for more tailored analysis that fits specific research questions or datasets. 
To utilize a custom statistical function, specify the argument `statFun = 'your_custom_function'` 
alongside the corresponding arguments required by your function. 

In [ ]:
pval,ES_obs,ES_null,_ = nest(statFun='custom_fun',args=custom_args,net_maps=net_7)